In [2]:
from google.colab import auth
from pprint import pprint
import time
import ee
import folium
import tensorflow as tf
from tensorflow import keras
import json


In [0]:
auth.authenticate_user()
ee.Authenticate()

In [3]:
ee.Initialize()
tf.enable_eager_execution()

EEException: ignored

In [0]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = l8sr.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapIdDict = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[45.4, -75.7])
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [0]:
# Sample the image at the points and add a random column.
sampleRegion = ee.Geometry.Rectangle([-139, 48.5, -114, 60])
sample = image.sample(region=sampleRegion, scale=30).randomColumn()

# Partition the sample approximately 70-30.
testing = sample.filter(ee.Filter.lt('random', 0.01))

In [0]:
# This is list of all the properties we want to export.
featureNames = list(bands)

# Create the tasks.
testingTask = ee.batch.Export.table.toDrive(
  collection=testing,
  description='Testing Export',
  fileFormat='csv',
  selectors=featureNames)

In [0]:
testingTask.start()

In [1]:
print(ee.batch.Task.list())

NameError: ignored

In [0]:
while testingTask.active():
  print('Polling for task (id: {}).'.format(testingTask.id))
  time.sleep(5)
print('Done with testing export.')

Polling for task (id: OINOFEWVU3ZRCPWGYDJEBGGX).


KeyboardInterrupt: ignored